In [1]:
import os
import keras
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

In [2]:
data = np.array([[0],[1]])
encoder = OneHotEncoder()
onehot_endcoder = encoder.fit(data)

In [13]:
# This cell updates result list for images with tumor

data = []
paths = []
result = []

for r, d, f in os.walk(r'D:/datasets/brain_tumour/brain_tumor_dataset/yes'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())

In [14]:
# This cell updates result list for images without tumor

paths = []
for r, d, f in os.walk(r'D:/datasets/brain_tumour/brain_tumor_dataset/no'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[1]]).toarray())

In [15]:
data = np.array(data)
data.shape

(139, 128, 128, 3)

In [16]:
result = np.array(result)
result = result.reshape(139,2)

In [17]:
print(f'Total number of images we have: {len(data)}')

Total number of images we have: 139


In [18]:
x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

In [19]:
print(f'Number of images in training data: {len(x_train)}')

Number of images in training data: 111


In [20]:
print(f'Number of images in testing data: {len(x_test)}')

Number of images in testing data: 28


In [21]:
x_train.shape

(111, 128, 128, 3)

In [22]:
y_train.shape

(111, 2)

In [23]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax')
print(model.summary())

D:\Anaconda\env\cmp_vision_env\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)                    │ (None, 128, 128, 32)        │             416 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_5 (Conv2D)                    │ (None, 128, 128, 32)        │           4,128 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_2                │ (None, 128, 128, 32)        │             128 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_2 (MaxPooling2D)       │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_3 (Dropout)                  │ (None, 64, 64, 32)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_6 (Conv2D)                    │ (None, 64, 64, 64)          │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_7 (Conv2D)                    │ (None, 64, 64, 64)          │          16,448 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_3                │ (None, 64, 64, 64)          │             256 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_3 (MaxPooling2D)       │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 32, 32, 64)          │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_1 (Flatten)                  │ (None, 65536)               │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 512)                 │      33,554,944 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_3 (Dense)                      │ (None, 2)                   │           1,026 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 33,585,602 (128.12 MB)

 Trainable params: 33,585,410 (128.12 MB)

 Non-trainable params: 192 (768.00 B)

None


In [24]:
y_train.shape

(111, 2)

In [25]:
history = model.fit(x_train, y_train, epochs = 30, batch_size = 40, verbose = 1,validation_data = (x_test, y_test))

Epoch 1/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 15s 3s/step - loss: 18.4446 - val_loss: 268.9442
Epoch 2/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 3s/step - loss: 25.1653 - val_loss: 32.9310
Epoch 3/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 15.7086 - val_loss: 56.2819
Epoch 4/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 4.8059 - val_loss: 79.2561
Epoch 5/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - loss: 5.2119 - val_loss: 40.6698
Epoch 6/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 11s 3s/step - loss: 2.6866 - val_loss: 11.7469
Epoch 7/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 3.0469 - val_loss: 13.5254
Epoch 8/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 3.7316 - val_loss: 9.6329
Epoch 9/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.8750 - val_loss: 18.0835
Epoch 10/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 1.4869 - val_loss: 20.0339
Epoch 11/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step - loss: 0.6847 - val_loss: 12.5150
Epoch 12/30
3/3 ━━━━━━━━━━━━━━━━━━━━ 9s 3s/step - loss: 0.3232 - val_loss: 7.2421
Ep